# Plasticity
<strong>J. M. Maas @ IfB, ETH Zürich, v. 2026-01-19</strong>

<div class='alert alert-info'>
    Select ▸▸ <strong>Restart the kernel and run all cells</strong> to run the widgets in this notebook and explore the plasticity data.
</div>

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import lmfit as lf

import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
from datetime import datetime

from pathlib import Path

# Import required modules
import utils.scleronomic_utils as scutils

The next cell imports the data points.

In [ ]:
# Import plasticity data
input_folder = Path('data/03_plasticity')

# Plasticity data
df_fits = pd.read_csv(input_folder / 'sample_plasticity_curves.csv', index_col=['sheet', 'sample', 'camera', 'component'])
df_increments = pd.read_csv(input_folder / 'plastic_increments.csv', index_col=['sheet', 'sample', 'camera', 'component', 'cycle'])

# Strength data
df_strength = pd.read_csv('data/02_strength/strength_data.csv', index_col=['sheet', 'sample'])

# Meta information
df_meta = pd.read_csv(input_folder / 'meta_information.csv', index_col=['sheet'])
df_orient = pd.read_csv(input_folder / 'sample_orientations.csv', header=[0, 1], index_col=[0,1])
df_moisture = pd.read_csv(input_folder / 'sample_moisture_contents.csv', index_col=['sample'])

The next cell visualizes the yield stress for a given offset yield strain.

In [ ]:
# Define offset stress visualization
df_export_offset = [pd.DataFrame()]

# Define camera parameters
camera_parameters = {  # key = constant_name, list = [loading_type, axial_direction, lateral_direction]
    'E_cL': ['compression', 'L', 'T'],
    'E_cR': ['compression', 'R', 'T'],
    'E_cT': ['compression', 'T', 'R'],
    'E_tL': ['tension', 'L', 'all'],
    'E_tR': ['tension', 'R', 'T'],
    'E_tT': ['tension', 'T', 'R'],
    'G_LR': ['shear', 'R', 'L'],
    'G_LT': ['shear', 'T', 'L'],
    'G_RL': ['shear', 'L', 'R'],
    'G_RT': ['shear', 'T', 'R'],
    'G_TL': ['shear', 'L', 'T'],
    'G_TR': ['shear', 'R', 'T']
}


# Define selection function
def select_by_condition(df, loading_type, axial_direction, lateral_direction, rh,
                        cameras=['cam1', 'cam2'], df_meta=df_meta, df_orient=df_orient):
    '''
    Select the data of a dataframe via specified anatomical conditions.
    Input:
        df                (pd.DataFrame) : DataFrame where the returned rows are select from.
                                           Must contain the index levels 'sample', 'camera', and 'component'.
        loading_type      (str)          : Select data by loading type. Options: 'compression', 'tension', 'shear'.
        axial_direction   (str)          : Axial direction of selected samples. Options: 'L', 'R', 'T'.
        lateral_direction (str)          : Lateral direction of selected samples. Options: 'L', 'R', 'T', 'all'.
        rh                (float)        : Relative humidity of the selected samples.
        cameras           (list of str)  : Optional. Enforce that only content of specific cameras is returned.
                                           Write the respected camera names into the list. Default: ['cam1', 'cam2'].
        df_meta           (pd.DataFrame) : Optional. DataFrame with the loading type, direction, and rh of the data sheets.
        df_orient         (pd.DataFrame) : Optional. DataFrame with the anatomical orientations of each sample.
    Output:
        df_out            (pd.DataFrame) : Part of the DataFrame df that fullfills the selected anatomical conditions.
    Example:
        df = select_by_condition(df_curves, 'compression', 'L', 'T', 65)
    '''
    # Initialize
    loading_type = loading_type.lower()
    axial_direction = axial_direction.upper()
    lateral_direction = lateral_direction.upper()
    
    # Select sheet
    direction = lateral_direction+axial_direction if loading_type == 'shear' else axial_direction
    sheets = df_meta.query(f'loading == "{loading_type}" and direction == "{direction}" and rh == {rh}').index
    if len(sheets) != 1:
        raise ValueError(f'Found no unique sheet for the selected conditions. Expected 1, but found {len(sheets):d}.')
    else:
        sheet = sheets[0]
    try:
        df = df.loc[sheet]
    except:
        print(f"Warning: No sheet '{sheet}' found in provided dataframe. Returning empty table.")
        return df.query(f"sheet == '{sheet}'")

    # Select camera sides
    df_orient = df_orient.loc[sheet].stack(0, future_stack=True)  # move first column multiindex level to new row multiindex level
    df_orient.index.rename(['sample', 'camera'], inplace=True)
    df_orient.query(f'camera in {cameras}', inplace=True)
    if lateral_direction != 'ALL': df_orient.query(f'x == "{lateral_direction}"', inplace=True)

    # Select valid indexes
    samples = df_orient.index.get_level_values('sample')
    cameras = df_orient.index.get_level_values('camera')
    component = 'exy_mean' if loading_type == 'shear' else 'eyy_mean'

    df_samples = df.index.get_level_values('sample')
    df_cameras = df.index.get_level_values('camera')
    df_components = df.index.get_level_values('component')

    indexes = [(sample, camera, component) for sample, camera in zip(samples, cameras)]
    selected_rows = [(sample, camera, component) in indexes for sample, camera, component in zip(df_samples, df_cameras, df_components)]

    # Crop dataframe
    df_out = df.loc[selected_rows]
    
    return df_out


# Define fitting function
def fit_yield_stress(df):
    # Initialize
    fit, R2 = None, 0.0

    # Extract data
    df = df.dropna(subset=['mc', 'sigma_yield'])
    mc = df['mc'].to_numpy() * 100
    max_stress = df['sigma_yield'].to_numpy()

    # Curve fitting
    f = lambda mc, p: np.poly1d(p)(mc)
    p0 = [-1, np.max(max_stress)]
    sol = scutils.lsq_fitting(f, xdata=mc, ydata=max_stress, p0=p0, method='lmfit', options=dict(maxiter=2000))
    
    # Return results
    fit = {
        'fun': f,
        'p': sol['x'],
        'perr': sol['err'],
        'success': sol['success'],
        'obj': sol['obj']
    }
    R2 = sol['R2']
    return fit, R2
    

# Define data collection function
def get_yield_stress(df_p, df_meta, loading_type, axial, lateral, p=0.2, threshold='max_stress', normalize=False):
    '''
    Determine the yield stress for each sample for a given loading type and anatomical direction.
    Returns only points where the yield stress is below the sample strength.
    Input:
        df_p         (pd.DataFrame) : Fitted data per sample with index levels 'sheet', 'sample',
                                      'camera, and 'component' and the columns 'K', 'n', and 'count'.
        loading_type (str)          : Applied loading type. Options: 'compression', 'tension', 'shear'.
        axial        (str)          : Axial direction of load application. Options: 'L', 'R', 'T'.
        lateral      (str)          : Lateral direction of used measurement data. Options: 'L', 'R', 'T', 'all'.
                                      Must not be the same value as axial.
        p            (float)        : Yield strain at determine yield stress.
        threshold    (str)          : Selects above which threshold samples are dropped from the data. Options:
                                      'none': Extrapolate all samples up to the selected strain p.
                                      'strength': Drop samples where R_p is above their respective strength.
                                      'max_stress': Drop samples where R_p is above their last measured stress.
        normalize    (bool)         : Optional. Normalize stresses by experimental strength, if True.
    Output:
        df_out       (pd.DataFrame) : Determined data points with moisture content 'mc' and yield stress 'fy'.
    '''
    # Initialize
    df_s = df_strength.copy()
    df_s.index = df_s.index.droplevel('sheet')
    
    # Iterate over all humdities
    f_ys, mcs, rhs, samples_all = [], [], [], []
    for rh in df_meta['rh'].unique():
        # Select data
        try:
            df = select_by_condition(df_p, loading_type, axial, lateral, rh, cameras=['cam1', 'cam2'])
        except:
            print(f"Warning: No data found for loading type '{loading_type}', axial '{axial}', lateral '{lateral}', and RH {rh}%.")
            continue

        # Select samples
        samples = df.index.get_level_values('sample')
        # samples = df.query(f"sample in {list(df_s.index.get_level_values('sample'))}").index.get_level_values('sample')  # only samples which are in df_strength
    
        # Get moisture content
        mc = df_moisture.loc[samples.str.upper(), 'moisture_content']
        mc = mc.fillna(mc.mean())  # fill missing values with average mc of that sample set
    
        # Determine yield stress
        K, n = df['K'], df['n']
        f_y = K * (p/100)**(1/n)  # yield stress
    
        # Keep only points where yield stress is below threshold
        max_stress = np.inf if threshold == 'none' else df[threshold].to_numpy()
        valid = np.where((f_y <= max_stress))[0]
        f_y, mc = f_y.iloc[valid], mc.iloc[valid]
        samples = samples[valid]

        # Remove outliers
        _, no_outliers = scutils.remove_outliers(f_y.to_numpy())

        # Normalize
        if normalize:
            strength = df['strength'].iloc[valid].to_numpy()  # true sample strength
            f_y = f_y / strength
    
        # Store result
        f_ys.append( f_y.iloc[no_outliers] )
        mcs.append( mc.iloc[no_outliers] )
        rhs.append( np.full(mcs[-1].shape, rh) )
        samples_all.append( samples[no_outliers] )
        
    # Assemble result
    fy = np.concatenate(f_ys)
    mc = np.concatenate(mcs)
    rh = np.concatenate(rhs)
    samples = np.concatenate(samples_all)
    
    df_out = pd.DataFrame(data={'rh': rh, 'mc': mc, 'sigma_yield': fy, 'eps_yield': p/100}, index=pd.Index(data=samples, name='sample'))
    return df_out


# Define plotting function
def plot_yield_stress_distribution(df, df_meta, constant, p, ax, threshold='max_stress', sigma=1, normalize=False, fit=True, lines='both'):
    '''
    Plot the yield stress distribution.
    Input:
        df        (pd.DataFrame) : Platicity parameters of fitted samples. Must contain index levels 'sheet',
                                   'sample', 'camera', and 'component', and the columns 'K' and 'n'.
        constant  (str)          : Constant to plot. Options: 'E_R', 'E_T', 'E_L', 'G_RT', 'G_RL', and 'G_TL'.
        p         (float)        : Plastic strain where yield stress is plotted. Can also be dict of floats,
                                   where the key matches to the value of constant.
        ax        (mpl.axes)     : Matplotlib axis object to draw the distribution into.
        threshold (str)          : Selects above which threshold samples are dropped from the data. Options:
                                   'none': Extrapolate all samples up to the selected strain p.
                                   'strength': Drop samples where R_p is above their respective strength.
                                   'max_stress': Drop samples where R_p is above their last measured stress.
        sigma     (int)          : Optional. Multiple of sigma width for confidence band and prediction band.
        normalize (bool)         : Optional. Normalize stresses by experimental strength, if True.
        fit       (bool)         : Optional. Show fits if True.
    Output:
        None
    '''
    # Initialize
    cparts = constant.split('_')
    if 'G' in constant:
        constants = [constant, f'{cparts[0]}_{cparts[1][::-1]}']
    else:
        constants = [constant.replace('_', '_c'), constant.replace('_', '_t')]
    rhs = df_meta['rh'].unique()
    if type(p) == dict:
        p = p_dict[constant]
    
    # Setup formatting
    colors = ['cornflowerblue', 'chocolate']
    markers = ['o', '^']
    fcs = [(1.0, 1.0, 1.0, 0.0), (1.0, 1.0, 1.0, 0.0)]

    # Iterate over all loading types
    lgd_labels, lgd_symbols, datas, fits, R2s = [], [], [], [], []
    for i, constant in enumerate(constants):
        if (lines=='blue' and i==1) or (lines=='red' and i==0):
            continue
        
        # Select constants
        loading_type, axial, lateral = camera_parameters[constant]
        direction = lateral+axial if loading_type == 'shear' else axial
        
        # Select data
        data = get_yield_stress(df, df_meta, loading_type, axial, lateral, p=p, normalize=normalize, threshold=threshold)
        datas.append( data )
        mc, fy = data['mc']*100, data['sigma_yield']
        
        # Plot data
        ax.scatter(mc, fy, zorder=3, marker=markers[i], color=colors[i], fc=fcs[i])
        
        # Format axis
        ax.set_xlim([0, 25])
        ax.grid(True)

        # Create fit
        if len(data) == 0 or not fit:
            continue
            
        fit, R2 = fit_yield_stress(data)
        fit['constant'] = constant
        fits.append( fit )
        R2s.append( R2 )

        # Plot fit
        fit_mc = np.linspace(0, 25, 100)
        fit_fy = fit['fun'](fit_mc, fit['p'])
        ax.plot(fit_mc, fit_fy, zorder=4, c=colors[i])

        # Plot error
        model = fit['obj']
        fit_confidence = model.eval_uncertainty(x=fit_mc, sigma=sigma)
        fit_prediction = model.dely_predicted
        ax.fill_between(fit_mc, fit_fy-fit_confidence, fit_fy+fit_confidence, zorder=2, color=colors[i], ec='none', alpha=0.5)
        ax.fill_between(fit_mc, fit_fy-fit_prediction, fit_fy+fit_prediction, zorder=1, color=colors[i], ec='none', alpha=0.3)
    
        # Create legend items
        lgd_labels.append( fr'$\sigma_{{ {loading_type[0]}, {direction} }}$ ($R^2 = {R2:.2f}$)' )
        lgd_symbols.append( plt.Line2D([0], [0], color=colors[i], lw=2, linestyle='-', marker=markers[i], mfc=fcs[i], mec=colors[i]) )

        # Return statistics
        # print(f'{constant}:\n  {fit["p"]}\n  {fit["perr"]}')

    # Add average
    data = pd.concat(datas)
    if not( len(data) == 0 or not fit ) and lines=='both':
        fit, R2 = fit_yield_stress(data)
        fit['constant'] = constant
        fits.append( fit )
        R2s.append( R2 )
        fit_mc = np.linspace(0, 25, 100)
        fit_fy = fit['fun'](fit_mc, fit['p'])
        ax.plot(fit_mc, fit_fy, zorder=5, c='#454545', ls='-.')

    # Format plot
    ax.set_xlabel(r'$\omega$ [%]')
    ax.set_ylabel(fr'$R_{{\mathrm{{p}} {p:.3f} }} / f_\mathrm{{max}}$ [-]' if normalize else fr'$R_{{\mathrm{{p}} {p:.3f} }}$ [MPa]')
    ax.legend(lgd_symbols, lgd_labels, loc='upper right', fontsize='small')

    # Return result
    df_export_offset[0] = data
    dfs_fits= []
    for fit in fits:
        p_col, p = zip( *[(f'p_{i}', p) for i, p in enumerate(fit['p'])] )
        err_col, err = zip( *[(f'Std_p_{i}', perr) for i, perr in enumerate(fit['perr'])] )
        dfs_fits.append( pd.DataFrame(
            data=dict( zip(list(p_col)+list(err_col), list(p)+list(err)) ),
            index=pd.Index([fit['constant']], name='component')
        ) )
    df_fit = pd.concat(dfs_fits)
    display(df_fit)


# Define visualization function
def visualize_offset_stress(df, df_meta, constant, show_lines, offset_strain, ax, threshold='max_stress', sigma=1, normalize=False):
    # Initialize
    ax.cla()

    # Create plot
    plot_yield_stress_distribution(df, df_meta, constant, p=offset_strain, ax=ax, threshold=threshold, sigma=sigma, normalize=normalize, fit=True, lines=show_lines)


# Export function
def export_csv_offset(button):
    if len(df_export_offset) > 0:
        export_path = f'export_yield_offsets_{w_constant_offset.value}.csv'
        df_export_offset[0].to_csv(export_path)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w_offset.children[-1]:
            w_offset.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path}.')
button_offset = widgets.Button(description='Export CSV', button_style='primary', icon='save')
button_offset.on_click(export_csv_offset)
display(button_offset)


# Create widgets
fig, ax = plt.subplots()
w_constant_offset = widgets.Dropdown(options=['E_R', 'E_T', 'E_L', 'G_RT', 'G_RL', 'G_TL'], description='component')
w_offset = widgets.interactive(
    visualize_offset_stress,
    df=widgets.fixed(df_fits),
    df_meta=widgets.fixed(df_meta),
    ax=widgets.fixed(ax),
    constant=w_constant_offset,
    show_lines=widgets.SelectionSlider(options=['both', 'blue', 'red']),
    offset_strain=widgets.FloatLogSlider(value=0.02, min=-3, max=1, step=0.001),
    threshold=widgets.Dropdown(value='strength', options=['none', 'max_stress', 'strength']),
    sigma=widgets.FloatSlider(value=1, min=0, max=3)
)
display(w_offset)

The next cell visualizes the plasticity curves constructed from those yield strains.

In [ ]:
# Define plasticity visualization
df_export_plasticity = [pd.DataFrame()]

# Define fitting function
def fit_rupture_strain(df):
    # Initialize
    fit, R2 = None, 0.0

    # Extract data
    df = df.dropna(subset=['mc', 'eps_max'])
    mc = df['mc'].to_numpy() * 100
    eps_max = df['eps_max'].to_numpy()

    # Curve fitting
    f = lambda mc, p: np.poly1d(p)(mc)
    p0 = [-1, np.max(eps_max)]
    sol = lsq_fitting(f, xdata=mc, ydata=eps_max, p0=p0, method='lmfit', options=dict(maxiter=2000))
    
    # Return results
    fit = {
        'fun': f,
        'p':  sol['x'],
        'success': sol['success'],
        'obj': sol['obj']
    }
    R2 = sol['R2']
    return fit, R2


# Extract rupture strains
def get_rupture_strain(df_p, loading_type, axial, lateral, loading_degree=0.97, ld_samples=['compression'], source='ro'):
    '''
    Determine the average plastic strain of a loading type at a given loading degree.
    Input:
        df_p           (pd.DataFrame) : Fitted data per sample with index levels 'sheet', 'sample',
                                        'camera, and 'component' and the columns 'K', 'n', and 'count'.
        loading_type   (str)          : Applied loading type. Options: 'compression', 'tension', 'shear'.
        axial          (str)          : Axial direction of load application. Options: 'L', 'R', 'T'.
        lateral        (str)          : Lateral direction of used measurement data. Options: 'L', 'R', 'T', 'all'.
                                        Must not be the same value as axial.
        loading_degree (float)        : Loading degree of the determined strain. Applies only for sample types
                                        listed in ld_samples.
        ld_samples     (list)         : List of samples the loading degree is applied on. Maximum strain for all
                                        other samples is rupture strain.
        source         (str)          : Data source of eps_max. Options: 'ro' for Ramberg-Osgood equation,
                                        'exp' for experimental maximum strain (ignores option loading_degree).
    Output:
        df_max_strains (pd.DataFrame) : Determined data points with strain 'eps_max' at stress 'sigma_max'.
    '''
    # Initialize
    df_s = df_strength.copy()
    df_s.index = df_s.index.droplevel('sheet')
    if not loading_type in ld_samples:
        loading_degree = 1.0  # take strain at maximum load if not compression sample
    df_exp = df_increments.abs().groupby('sample')[['x', 'y']].max()
    
    # Iterate over relative humidity
    rhs = sorted(df_meta['rh'].unique())
    dfs = []
    for rh in rhs:
        '''
        # Select experimental plastic data
        df = select_by_condition(df_increments, loading_type, axial, lateral, rh, cameras=['cam1', 'cam2'])
        samples = df.index.droplevel(-1).unique().get_level_values('sample')
        eps_max = df['x'].abs().groupby(['sample', 'camera', 'component']).max()
        sigma_max = df['y'].abs().groupby(['sample', 'camera', 'component']).max()
        '''
    
        # Select fitted plastic strain
        df = select_by_condition(df_p, loading_type, axial, lateral, rh, cameras=['cam1', 'cam2'])
        samples = df.index.get_level_values('sample')
        match source:
            case 'ro':  # from Ramberg-Osgood equation
                # sigma_max = loading_degree*df_s.loc[samples, 'max_stress'].to_numpy()
                sigma_max = loading_degree*df['strength'].to_numpy()
                eps_max = (sigma_max / df['K'])**df['n']  # calculate strain at given stress with inverse Ramberg-Osgood equation
            case 'exp':  # from experimental plastic increments
                sigma_max = df_exp.loc[samples, 'y'].to_numpy()
                eps_max = df_exp.loc[samples, 'x']
            case _:
                raise ValueError(f'Invalid source option "{source}".')
    
        # Remove outliers
        for i in range(1):
            _, no_outliers = scutils.remove_outliers(eps_max.to_numpy(), f=1.5)
            sigma_max = sigma_max[no_outliers]
            eps_max = eps_max.iloc[no_outliers]
            samples = samples[no_outliers]
    
        # Get moisture content
        samples_mc = samples.str.upper()
        mc = df_moisture.loc[samples_mc, 'moisture_content']
        mc = mc.fillna(mc.mean())  # fill missing values with average mc of that sample set
    
        # Store data
        data = pd.DataFrame(data={'eps_max': eps_max, 'sigma_max': sigma_max})
        data['mc'] = mc.to_numpy()
        data['rh'] = rh
        dfs.append(  data )
    
    # Assemble result
    df_max_strains = pd.concat(dfs)
    return df_max_strains

# Explore parameters
def explore_plastic_parameters(df, constant, n, loading_degree, ax, show_uncertainty=True, normalize=False, threshold='strength'):
    '''
    Input:
        constant       (str)   : Engineering constant.
        loading_degree (float) : Loading degree <= 1.0 of ultimate strain.
        n              (int)   : Number of reference points.
        normalize      (bool)  : Whether to normalize the stress data by the sample strength.
        omit_failed    (bool)  : Remove data points where sample yield stress is higher then
                                 the last measured sample stress if True (default).
    '''
    # Initialize
    print()  # clear error messages
    if not ax is None:
        ax.cla()
    colors = ['gold', 'mediumseagreen', 'cornflowerblue', 'slateblue']
    
    # Get settings
    loading_type, axial, lateral = camera_parameters[constant]
    direction = lateral+axial if loading_type == 'shear' else axial
    # df = df.query('sheet != "data_4_TT"')  # line data
    
    # Collect rupture parameters
    df_eps_ult = get_rupture_strain(df, loading_type, axial, lateral, loading_degree=loading_degree, ld_samples=['compression', 'tension', 'shear'])
    fun_eps_ult = lambda mc: df_eps_ult['eps_max'].mean()
    rhs = sorted( df_eps_ult['rh'].unique() )
    mcs = np.array( [df_eps_ult.query(f'rh == {rh}')['mc'].mean() * 100 for rh in rhs] )  # moisture contents
    eps_ult = np.array( [fun_eps_ult(mc) for mc in mcs] )  # rupture strains at loading degree
    
    # Collect yield parameters
    eps_yield = np.geomspace(eps_ult[-1]/(5*n), eps_ult[-1], n)  # omit zero-point
    fits_R_p = []
    for eps in eps_yield:  # get yield point for every prescribed strain
        p = eps * 100
        df_R_p = get_yield_stress(df, df_meta, loading_type, axial, lateral, p=p, normalize=normalize, threshold=threshold)
        fit_R_p, R2_R_p = fit_yield_stress(df_R_p)
        fits_R_p.append( fit_R_p )
    
    # Determine plasticity law
    dfs_yield, dfs_fit = [], []
    for i, mc in enumerate(mcs):
        # Get yield points
        sigma_yield, sigma_err = [], []
        for eps, fit_R_p in zip(eps_yield, fits_R_p):
            fun_sigma_yield = lambda mc: fit_R_p['fun'](mc, fit_R_p['p'])
            conf = fit_R_p['obj'].eval_uncertainty(x=eps, sigma=1)
            pred = fit_R_p['obj'].dely_predicted
            sigma_yield.append( fun_sigma_yield(mc) )
            sigma_err.append( pred )
        sigma_yield = np.array( sigma_yield )
        sigma_err = np.array( sigma_err )
    
        # Fit plasticity law
        f = lambda eps, n, K: eps**(1/n) * K
        model = lf.Model(f)
        result = model.fit(sigma_yield, eps=eps_yield, n=1, K=20)
        sol = result.best_values
        n, K = sol['n'], sol['K']
        dfs_fit.append( pd.DataFrame( data={'n': n, 'K': K}, index=pd.Index([f'{mc:.2f}'], name='moisture_content') ) )

        if not ax is None:
            # Plot plasticity
            eps_fit = np.linspace(0, eps_ult[i], 200)
            sigma_fit = eps_fit**(1/n) * K
            ax.plot(eps_fit, sigma_fit, label=rf'$\varphi = {rhs[i]:.0f}\%$ ($\omega = {mc:.2f}\%$)', color=colors[i], zorder=5)
            ax.scatter(eps_yield, sigma_yield, color=colors[i], marker='x', zorder=10)
            if show_uncertainty:
                ax.fill_between(eps_yield, sigma_yield-sigma_err, sigma_yield+sigma_err, color=colors[i], alpha=0.3, zorder=4)

        # Store yield data
        dfs_yield.append(
            pd.DataFrame(
                data={
                    'stress': sigma_yield,
                    'stress_error': sigma_err,
                },
                index=pd.MultiIndex.from_tuples([(mc, eps) for eps in eps_yield], names=['moisture_content', 'strain'])
            )
        )

    if not ax is None:
        # Format plot
        ax.set_xlabel('Plastic strain [-]')
        ax.set_ylabel('Stress [MPa]')
        ax.set_xlim([0, 1.05*eps_ult[-1]])
        ax.legend()
        ax.grid(True)

    # Export result
    df_yield = pd.concat(dfs_yield)
    df_export_plasticity[0] = df_yield
    df_fit = pd.concat(dfs_fit)
    display(df_fit)
    return df_yield


# Define visualization function
def visualize_plasticity(df, constant, n, loading_degree, ax, show_uncertainty=True, normalize=False, threshold='strength'):
    explore_plastic_parameters(df, constant, n, loading_degree, ax, show_uncertainty=True, normalize=False, threshold='strength')


# Export function
def export_csv_plasticity(button):
    if len(df_export_plasticity) > 0:
        export_path = f'export_plasticity_{w_plasticity_component.value}.csv'
        df_export_plasticity[0].to_csv(export_path)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w_plasticity.children[-1]:
            w_plasticity.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path}.')
button_plasticity = widgets.Button(description='Export CSV', button_style='primary', icon='save')
button_plasticity.on_click(export_csv_plasticity)
display(button_plasticity)


# Create widget
fig, ax = plt.subplots()
w_plasticity_component = widgets.Dropdown(options=sorted(camera_parameters.keys()), description='component')
w_plasticity = widgets.interactive(
    visualize_plasticity,
    df=widgets.fixed(df_fits),
    constant=w_plasticity_component,
    n=widgets.IntSlider(value=10, min=2, max=20),
    loading_degree=widgets.FloatSlider(value=0.97, min=0.01, max=1, step=0.01, readout_format='.2f'),
    ax=widgets.fixed(ax),
    threshold=['none', 'max_stress', 'strength']
)
fig.tight_layout()
display(w_plasticity)

## References
---
<ul>
    <li>Ramberg, W., Osgood, W.R., 1943. Description of stress-strain curves by three parameters (Technical Note No. 902). National Advisory Committee for Aeronautics, Washington, DC.</li>
</ul>